# Aims volume orientation manipulation and nibabel

In Aims 5.2, indices axes orientation, memory layout and storage (disk) layout are known and can be manipulated and changed. This allows to use the axes we want, and convert from/to nibabel objects.

For a complete documentation, please read https://brainvisa.info/cartodata/doxygen/cartovolumes.html#volume_orient

## imports

In [16]:
from soma import aims
import nibabel
import tempfile
import os

## load a volume using both aims and niabel

This will allow to compare

In [2]:
image_path = aims.carto.Paths.findResourceFile('anatomical_templates/MNI152_T1_1mm.nii.gz')
avol = aims.read(image_path)
nvol = nibabel.load(image_path)

In [3]:
print('aims    shape:', avol.shape)
print('nibabel shape:', nvol.shape)
print('aims voxel (100, 102, 62):', avol[100, 102, 62, 0])
print('nibabel:                  ', nvol.get_fdata()[100, 102, 62])

aims    shape: (182, 218, 182, 1)
nibabel shape: (182, 218, 182)
aims voxel (100, 102, 62): 5933.0
nibabel:                   6464.0


As we see, voxels values are **not the same**: axes are actually flipped.
Aims reads data in LPI orientation.
Nibabel reads data in disk storage orientation (which thus depnds on haw data has been saved).

## Check AIMS orientation and disk storage orientation

In [4]:
storage_orient_v = avol.storageLayoutOrientation()
storage_orient = avol.referential().orientationStr(storage_orient_v)
print('disk storage orientation:', storage_orient)
print('AIMS volume orientation:', avol.referential().orientationStr())

disk storage orientation: LAST
AIMS volume orientation: LPIT


## Building an AIMS volume from nibabel data

We must set the correct nibabel orientation on the volume, which is storage orientation.

In [5]:
avol2 = aims.Volume(nvol.get_fdata())
avol2.referential().setOrientation(storage_orient)
print('aims from nibabel:\norientation:', avol2.referential().orientationStr())
print('shape:', avol2.shape)

aims from nibabel:
orientation: LAST
shape: (182, 218, 182, 1)


In [9]:
print('aims from nibabel:', avol2[100, 102, 62, 0], "==", nvol.get_fdata()[100, 102, 62], '(nibabel)')
print('                          !=', avol[100, 102, 62, 0], '(aims)')

aims from nibabel: 6464.0 == 6464.0 (nibabel)
                          != 5933.0 (aims)


As we are in storage orientation, we should use indices as in nibabel, which actually works !

### Flipping to another orientation

If we want to go back to the default AIMS orientation (LPI) without copying the voxels data, it is possible:

In [13]:
avol2.flipToOrientation('LPI')
print('in LPI orientation:', avol2[100, 102, 62, 0], '==', avol[100, 102, 62, 0], '(aims)')

in LPI orientation: 5933.0 == 5933.0 (aims)


## Building a Nibabel volume from AIMS data

First let's see what is wrong when doing things too quickly...

In [15]:
nvol2 = nibabel.Nifti1Image(avol.np, None)
print('nibabel from aims (LPI):\nshape:', nvol2.shape)
print(nvol2.get_fdata()[100, 102, 62, 0], '!WARNING! matches aims but is wrong in nibabel orientation')

nibabel from aims (LPI):
shape: (182, 218, 182, 1)
5933.0 !WARNING! matches aims but is wrong in nibabel orientation


Actually the LPI-oriented array is used directly in nibabel. *This is not wrong in itself*, but nibabel does not know it is LPI, so saving the image will result in a flipped image.

In [18]:
tfile = tempfile.mkstemp(prefix='aims_volume_orient', suffix='.nii')
tfilename = tfile[1]
os.close(tfile[0])
print('saving to:', tfilename)

saving to: /tmp/aims_volume_orientrjs5q6vi.nii


In [22]:
nibabel.save(nvol2, tfilename)
nvol3 = nibabel.load(tfilename)
print('after nibabel save/reload:', nvol3.get_fdata()[100, 102, 62], '!=', 
      nvol.get_fdata()[100, 102, 62], '(nibabel)')

after nibabel save/reload: [5933.] != 6464.0 (nibabel)


The reloaded image is not like the original one: it is flipped since the orientation does not match the expected default one.
Alternatively, providing an affine to nibabel.Nifti1Image could fix it. Let's use the aims orientation manipulation instead.

Cleanup the temporary file, by the way...

In [27]:
os.unlink(tfilename)

If we flip the AIMS volume to storage orientation before buildig the nibabel image from it, thigs are going right.

In [25]:
avol.flipToOrientation(storage_orient)
nvol2 = nibabel.Nifti1Image(avol.np, None)
print('nibabel from aims (storage):\nshape:', nvol2.shape)
print(nvol2.get_fdata()[100, 102, 62, 0], '==', nvol.get_fdata()[100, 102, 62], '(nibabel) *CORRECT!*')

nibabel from aims (storage):
shape: (182, 218, 182, 1)
6464.0 == 6464.0 (nibabel) *CORRECT!*
